In [1]:
import argparse
import gzip
import os
import sys
import time
import urllib

import numpy
import tensorflow as tf

from six.moves import xrange

In [2]:
SOURCE_URL = 'http://yann.lecun.com/exdb/mnist/'
WORK_DIRECTORY = 'data'
IMAGE_SIZE = 28
NUM_CHANNELS = 1
PIXEL_DEPTH = 255
NUM_LABELS = 10
VALIDATION_SIZE = 5000  # Size of the validation set.
SEED = 66478  # Set to None for random seed.
BATCH_SIZE = 1
NUM_EPOCHS = 100
EVAL_BATCH_SIZE = 1
EVAL_FREQUENCY = 5000  # Number of steps between evaluations.

EMBEDDING_SIZE = 8
NUM_UNROLL_STEPS = 3

In [3]:
def maybe_download(filename):
  """Download the data from Yann's website, unless it's already here."""
  if not tf.gfile.Exists(WORK_DIRECTORY):
    tf.gfile.MakeDirs(WORK_DIRECTORY)
  filepath = os.path.join(WORK_DIRECTORY, filename)
  if not tf.gfile.Exists(filepath):
    filepath, _ = urllib.request.urlretrieve(SOURCE_URL + filename, filepath)
    with tf.gfile.GFile(filepath) as f:
      size = f.size()
    print('Successfully downloaded', filename, size, 'bytes.')
  return filepath


def extract_data(filename, num_images):
  """Extract the images into a 4D tensor [image index, y, x, channels].

  Values are rescaled from [0, 255] down to [-0.5, 0.5].
  """
  print('Extracting', filename)
  with gzip.open(filename) as bytestream:
    bytestream.read(16)
    buf = bytestream.read(IMAGE_SIZE * IMAGE_SIZE * num_images * NUM_CHANNELS)
    data = numpy.frombuffer(buf, dtype=numpy.uint8).astype(numpy.float32)
    data = (data - (PIXEL_DEPTH / 2.0)) / PIXEL_DEPTH
    data = data.reshape(num_images, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS)
    return data


def extract_labels(filename, num_images):
  """Extract the labels into a vector of int64 label IDs."""
  print('Extracting', filename)
  with gzip.open(filename) as bytestream:
    bytestream.read(8)
    buf = bytestream.read(1 * num_images)
    labels = numpy.frombuffer(buf, dtype=numpy.uint8).astype(numpy.int64)
  return labels

from collections import Counter

def error_rate(predictions, labels):
  """Return the error rate based on dense predictions and sparse labels."""
  argmax = numpy.argmax(predictions, 1)
  print(Counter(argmax).most_common())
  return 100.0 - (
      100.0 *
      numpy.sum(argmax == labels) /
      predictions.shape[0])

train_data_filename = maybe_download('train-images-idx3-ubyte.gz')
train_labels_filename = maybe_download('train-labels-idx1-ubyte.gz')
test_data_filename = maybe_download('t10k-images-idx3-ubyte.gz')
test_labels_filename = maybe_download('t10k-labels-idx1-ubyte.gz')

# Extract it into numpy arrays.
train_data = extract_data(train_data_filename, 60000)
train_labels = extract_labels(train_labels_filename, 60000)
test_data = extract_data(test_data_filename, 10000)
test_labels = extract_labels(test_labels_filename, 10000)

# Generate a validation set.
validation_data = train_data[:VALIDATION_SIZE, ...]
validation_labels = train_labels[:VALIDATION_SIZE]
train_data = train_data[VALIDATION_SIZE:, ...]
train_labels = train_labels[VALIDATION_SIZE:]
train_size = train_labels.shape[0]

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [4]:
def model_step(input_image, prior, train=False):
    """The Model definition."""
    assert input_image.get_shape()[0] == 1, "HyperNet operates on single images only"
    
    prior_embeddings = tf.get_variable("prior_embeddings",
                                        shape=[NUM_LABELS, EMBEDDING_SIZE],
                                        initializer=tf.random_uniform_initializer(
                                            minval=-1.0/numpy.sqrt(NUM_LABELS), maxval=1.0/numpy.sqrt(NUM_LABELS)))
    embedding_features = tf.matmul(prior, prior_embeddings)
    for i in range(4):
        embedding_features = embedding_features + tf.layers.dense(
            embedding_features, EMBEDDING_SIZE, use_bias=False, activation=tf.nn.relu)
    
    def get_dynamic_weights(weights_shape):
        num_weights = numpy.prod(weights_shape)
        dynamic_weights_flat = tf.layers.dense(embedding_features, num_weights)
        dynamic_weights = tf.reshape(dynamic_weights_flat, weights_shape)
        dynamic_weights.set_shape(weights_shape)
        return dynamic_weights
    
    conv1_weights = get_dynamic_weights([5, 5, NUM_CHANNELS, 4])
    conv1_biases = get_dynamic_weights([4])
    conv2_weights = get_dynamic_weights([5, 5, 4, 8])
    conv2_biases = get_dynamic_weights([8])
    fc2_weights = get_dynamic_weights([8, NUM_LABELS])
    fc2_biases = get_dynamic_weights([NUM_LABELS])
    
    conv = tf.nn.conv2d(input_image,
                        conv1_weights,
                        strides=[1, 1, 1, 1],
                        padding='SAME')
    # Bias and rectified linear non-linearity.
    relu = tf.nn.relu(tf.nn.bias_add(conv, conv1_biases))
    # Max pooling. The kernel size spec {ksize} also follows the layout of
    # the data. Here we have a pooling window of 2, and a stride of 2.
    pool = tf.nn.max_pool(relu,
                          ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1],
                          padding='SAME')
    conv = tf.nn.conv2d(pool,
                        conv2_weights,
                        strides=[1, 1, 1, 1],
                        padding='SAME')
    relu = tf.nn.relu(tf.nn.bias_add(conv, conv2_biases))
    pool = tf.nn.max_pool(relu,
                          ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1],
                          padding='SAME')
    pool_shape = pool.get_shape()
    hidden = tf.nn.avg_pool(pool, ksize=[1, pool_shape[1], pool_shape[2], 1],
                            strides=[1, 1, 1, 1], padding="VALID")
    hidden = tf.squeeze(hidden, axis=[1, 2])
    # Add a 50% dropout during training only. Dropout also scales
    # activations such that no rescaling is needed at evaluation time.
    #if train:
    #    hidden = tf.nn.dropout(hidden, 0.5, seed=SEED)
    logits = tf.matmul(hidden, fc2_weights) + fc2_biases
    posteriors = tf.nn.softmax(logits)
    return logits, posteriors

def apply(input_image, train):
    priors = numpy.array([[1/NUM_LABELS for _ in range(NUM_LABELS)]], dtype=numpy.float32)
    results = []
    loss = 0.0
    for step in range(NUM_UNROLL_STEPS):
        logits, posteriors = model_step(input_image, priors, train=train)
        priors = posteriors
        results.append((logits, posteriors))
        loss += tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
              labels=train_labels_node, logits=logits))
    return logits, loss

In [ ]:
train_data_node = tf.placeholder(
    tf.float32,
    shape=(BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS))
train_labels_node = tf.placeholder(tf.int64, shape=(BATCH_SIZE,))
eval_data = tf.placeholder(
    tf.float32,
    shape=(EVAL_BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS))


# Optimizer: set up a variable that's incremented once per batch and
# controls the learning rate decay.
batch = tf.Variable(0, dtype=tf.float32)
# Decay once per epoch, using an exponential schedule starting at 0.01.
learning_rate = tf.train.exponential_decay(
    1e-3,                # Base learning rate.
    batch * BATCH_SIZE,  # Current index into the dataset.
    train_size,          # Decay step.
    0.95,                # Decay rate.
    staircase=True)

# Predictions for the current training minibatch.
with tf.variable_scope("model", reuse=tf.AUTO_REUSE):
    train_prediction, loss = apply(train_data_node, train=True)

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss,
                                                   global_step=batch)


# Predictions for the test and validation, which we'll compute less often.
with tf.variable_scope("model", reuse=True):
    eval_prediction, _ = apply(eval_data, train=False)

  # Small utility function to evaluate a dataset by feeding batches of data to
  # {eval_data} and pulling the results from {eval_predictions}.
  # Saves memory and enables this to run on smaller GPUs.
def eval_in_batches(data, sess):
    """Get all predictions for a dataset by running it in small batches."""
    size = data.shape[0]
    if size < EVAL_BATCH_SIZE:
      raise ValueError("batch size for evals larger than dataset: %d" % size)
    predictions = numpy.ndarray(shape=(size, NUM_LABELS), dtype=numpy.float32)
    for begin in xrange(0, size, EVAL_BATCH_SIZE):
      end = begin + EVAL_BATCH_SIZE
      if end <= size:
        predictions[begin:end, :] = sess.run(
            eval_prediction,
            feed_dict={eval_data: data[begin:end, ...]})
      else:
        batch_predictions = sess.run(
            eval_prediction,
            feed_dict={eval_data: data[-EVAL_BATCH_SIZE:, ...]})
        predictions[begin:, :] = batch_predictions[begin - size:, :]
    return predictions

  # Create a local session to run the training.
start_time = time.time()
with tf.Session() as sess:
    # Run all the initializers to prepare the trainable parameters.
    tf.global_variables_initializer().run()
    print('Initialized!')
    # Loop through training steps.
    for step in xrange(int(NUM_EPOCHS * train_size) // BATCH_SIZE):
      # Compute the offset of the current minibatch in the data.
      # Note that we could use better randomization across epochs.
      offset = (step * BATCH_SIZE) % (train_size - BATCH_SIZE)
      batch_data = train_data[offset:(offset + BATCH_SIZE), ...]
      batch_labels = train_labels[offset:(offset + BATCH_SIZE)]
      # This dictionary maps the batch data (as a numpy array) to the
      # node in the graph it should be fed to.
      feed_dict = {train_data_node: batch_data,
                   train_labels_node: batch_labels}
      # Run the optimizer to update weights.
      sess.run(optimizer, feed_dict=feed_dict)
      # print some extra information once reach the evaluation frequency
      if step % EVAL_FREQUENCY == 0:
        # fetch some extra nodes' data
        l, lr, predictions = sess.run([loss, learning_rate, train_prediction],
                                      feed_dict=feed_dict)
        elapsed_time = time.time() - start_time
        start_time = time.time()
        print('Step %d (epoch %.2f), %.1f ms' %
              (step, float(step) * BATCH_SIZE / train_size,
               1000 * elapsed_time / EVAL_FREQUENCY))
        print('Minibatch loss: %.3f, learning rate: %.6f' % (l, lr))
        print('Minibatch error: %.1f%%' % error_rate(predictions, batch_labels))
        print('Validation error: %.1f%%' % error_rate(
            eval_in_batches(validation_data, sess), validation_labels))
        sys.stdout.flush()
    # Finally print the result!
    test_error = error_rate(eval_in_batches(test_data, sess), test_labels)
    print('Test error: %.1f%%' % test_error)


Initialized!
Step 0 (epoch 0.00), 0.4 ms
Minibatch loss: 6.671, learning rate: 0.001000
[(1, 1)]
Minibatch error: 100.0%
[(1, 5000)]
Validation error: 88.7%
Step 5000 (epoch 0.09), 9.4 ms
Minibatch loss: 3.105, learning rate: 0.001000
[(3, 1)]
Minibatch error: 0.0%
[(0, 736), (8, 606), (4, 543), (7, 537), (1, 527), (6, 471), (9, 469), (3, 413), (5, 395), (2, 303)]
Validation error: 17.2%
Step 10000 (epoch 0.18), 9.2 ms
Minibatch loss: 0.448, learning rate: 0.001000
[(5, 1)]
Minibatch error: 0.0%
[(6, 560), (4, 535), (1, 531), (8, 521), (9, 511), (2, 500), (7, 498), (5, 491), (0, 480), (3, 373)]
Validation error: 10.7%
Step 15000 (epoch 0.27), 9.5 ms
Minibatch loss: 0.194, learning rate: 0.001000
[(5, 1)]
Minibatch error: 0.0%
[(1, 589), (9, 550), (5, 535), (6, 534), (8, 491), (4, 489), (0, 478), (3, 462), (7, 458), (2, 414)]
Validation error: 9.1%
Step 20000 (epoch 0.36), 9.5 ms
Minibatch loss: 1.548, learning rate: 0.001000
[(3, 1)]
Minibatch error: 0.0%
[(7, 586), (1, 543), (4, 527),